In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import pandas as pd

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to extract most important sentences based on word importance scores
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Hapus paragraf kosong
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)  # Pisah paragraf menjadi kalimat
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            # Hapus token_type_ids jika ada
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            # Cek jika scores tidak kosong
            if scores.numel() > 0:  # Memastikan tensor tidak kosong
                k = min(5, scores.size(0))  # Set k tidak lebih besar dari ukuran tensor
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                # Jika kalimat memiliki kata-kata penting, tambahkan ke ringkasan paragraf
                if len(top_words) > 0:
                    paragraph_summary.append(sentence)  # Menambahkan kalimat ke ringkasan paragraf

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        # Kombinasikan kalimat paling penting dari setiap paragraf
        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])  # Ambil kalimat paling penting dari setiap paragraf

    # Gabungkan kalimat yang dipilih menjadi ringkasan singkat
    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find(title_tag, class_=title_class).get_text(strip=True)
    date = soup.find(date_tag, class_=date_class).get_text(strip=True)
    content = soup.find(content_tag, class_=content_class).get_text(strip=True)

    # Automatically summarize the content using BERTSUM
    summary = summarize_text(content, model, tokenizer, num_sentences=1)  # Adjust to only 1 sentence per paragraph

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://www.cnbcindonesia.com/market/20240819141519-17-564451/ojk-bekukan-1001-entitas-ilegal-850-ternyata-pinjol-ilegal',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240819080903-17-564272/bitcoin-diguyur-banyak-sentimen-positif-pasar-kripto-malah-merah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240817185527-17-564106/kisah-negara-kecil-mendadak-kaya-raya-berkat-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/entrepreneur/20240815205416-25-563433/ngenes-sosok-ini-dapat-durian-runtuh-rp163-m-tapi-berakhir-dipenjara',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240816053707-17-563440/kripto-turun-berjamaah-pasar-khawatir-hal-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240815083932-17-563184/pemerintah-as-pindahkan-10000-btc-pasar-kripto-kena-syok',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240814163131-17-563084/sempat-turun-ke-us--49000-bitcoin-bisa-jaga-tren-kenaikan-harga',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240814052344-17-562810/etf-ethereum-spot-catat-inflow-rp80-miliar-pasar-kripto-kompak-hijau',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240813153501-19-562725/buka-bukaan-mirza-soal-kondisi-asuransi-kesiapan-ojk-awasi-kripto-cs',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240813152254-17-562720/skema-ponzi-penipuan-kripto-rp-10-t-regulator-as-gugat-novatech',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240813124405-17-562649/crypto-wallet-ini-tembus-30-juta-pengguna-naik-50-dalam-4-bulan',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240813064639-17-562500/bos-bursa-blak-blakan-penyebab-ihsg-kurang-tenaga-tahun-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240813052433-17-562488/paus-ini-setor-rp-200-miliar-eth-pasar-kripto-sambut-positif',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240812190458-8-562463/video-prabowo-janji-lanjutkan-ikn-hingga-bandar-kripto-bangkrut',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240812131825-128-562331/seperti-sulap-harga-bitcoin-terbang-ratusan-juta-dalam-sekejap',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240812093812-17-562217/bandar-kripto-bangkrut-diminta-bayar-denda-rp-203-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240812072705-17-562157/traditional-financee-defi-jadi-ancaman-pasar-kripto-dibuka-merah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240810113838-17-561950/pejabat-ojk-blak-blakan-pernah-jadi-korban-penipuan-lewat-instagram',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809163011-17-561849/cek-beda-rupiah-digital-dan-uang-elektronik',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809131958-17-561745/kripto-bisa-jadi-agunan-bank-ini-kata-bos-ojk',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809124749-17-561733/banyak-dibilang-kemahalan-ojk-buka-wacana-sesuaikan-pajak-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809122836-17-561726/setelah-jadi-pengawas-kripto-ojk-akan-lakukan-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809121333-17-561719/bos-ojk-beberkan-soal-formasi-anggota-anti-scam-center',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240809100550-37-561677/bandar-kripto-bangkrut-mau-bagi-bagi-duit-rp-202-t',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809105854-17-561700/ojk-bikin-peta-jalan-kripto-4-isu-ini-jadi-fokus-utama',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240809061224-17-561606/rusia-legalkan-penambangan-kripto-bitcoin-cs-kompak-menguat',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240809065116-128-561619/agenda-hari-ini-jumat-9-agustus-ada-peluncuran-peta-jalan-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240809051849-37-561598/rupiah-digital-segera-terbit-bi-ungkap-jadwalnya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240808151602-37-561440/bitcoin-buatan-cia-fakta-bikin-kaget-diungkap-sahabat-donald-trump',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240808074925-17-561272/ikuti-bursa-as-pasar-kripto-lagi-lagi-kompak-ambruk',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240807071707-17-560966/gara-gara-bitcoin-negara-kecil-ini-jadi-terkaya-di-dunia',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240807060900-17-560952/investor-institusi-blackrock-hold-bitcoin-pasar-kripto-menguat',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240806100126-131-560694/investor-saham-ri-tembus-jutaan-tapi-malas-transaksi-gak-punya-uang',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240806075625-17-560653/bitcoin-cs-mulai-rebound-pasar-kripto-ada-harapan',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240805175942-8-560571/video-ekonomi-ri-tumbuh-5-05-hingga-bitcoin-ethereum-ambruk',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805160108-17-560524/saham-mulai-ditinggalkan-investor-lari-ke-kripto-bos-ojk-buka-suara',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805151001-17-560506/transaksi-kripto-ri-tembus-rp-30175-t-di-2024-naik-3-kali-lipat',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805110919-17-560383/bitcoin-ethereum-ambruk-rp-4400-triliun-lenyap-dalam-24-jam',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805105000-17-560368/wajib-baca-ini-perbedaan-rupiah-digital-dan-uang-elektronik',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805092700-17-560335/ini-alasan-bitcoin-ambruk-ke-us--53000-ethereum-anjlok-20',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240805062732-17-560280/gonjang-ganjing-as-goyang-pasar-kripto-bitcoin-ambruk-ke-us--58000',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240804161442-17-560232/berkat-bitcoin-negara-kecil-ini-jadi-yang-paling-kaya-di-dunia',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240801081525-128-559404/investor-saham-ri-tembus-jutaan-tapi-malas-transaksi-gak-punya-uang',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240802060402-17-559734/pertemuan-sec-dan-xrp--ripple--bikin-pasar-kripto-memerah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240801051741-17-559373/the-fed-tahan-suku-bunga-bitcoin-cs-langsung-turun',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240731045805-17-559016/keputusan-3-bank-sentral-dinanti-pasar-kripto-balik-melemah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240730131449-37-558825/efek-donald-trump-bitcoin-meroket-ke-rp-11-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240730112025-128-558757/sederet-ambisi-besar-donald-trump-untuk-kripto-bitcoin-siap-terbang',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240730074018-128-558670/klarifikasi-nanovest-atas-kerugian-nasabah-rp-1-m',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240730072019-17-558659/pemerintah-as-pindahkan-rp324-triliun-kripto-di-zona-merah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240729105122-37-558380/takut-sama-china-donald-trump-mendadak-mau-timbun-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240729061118-128-558280/transaksi-bursa-kripto-ri-bermasalah-ada-nasabah-rugi-sampai-rp1-m',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240729081044-17-558308/dibanding-bitcoin-warren-buffett-pilih-investasi-lahan-pertanian',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240729070503-17-558295/donald-trump-janjikan-hal-ini-pasar-kripto-berfluktuasi',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240728055303-17-558161/trump-dukung-kripto-bitcoin-langsung-loyo',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240727082523-17-558079/konferensi-bitcoin-semakin-dekat-pasar-kripto-sambut-euforia',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240726193845-37-558046/langgar-perjanjian-raja-kripto-dunia-didenda-rp-73-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240726130925-37-557879/cari-sandi-harian-hamster-kombat-awas-malah-rekening-terkuras',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240726102824-17-557812/donasi-trump-tembus-rp48-miliar-bitcoin-cs-kompak-terbang',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240724182606-128-557366/tabungan-warga-ri-terkuras-buat-judol-bantu-pemerintah-beli-beras',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240726061229-17-557742/aksi-jual-etf-ethereum-bikin-kripto-loyo-tak-bertenaga',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240725172414-17-557667/ethereum-etf-meluncur-bawa-kabar-baik-buat-investor-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240725151436-17-557619/transaksi-kripto-di-ri-capai-rp-30175-t-naik-35417',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240725142946-37-557599/aplikasi-pengganti-whatsapp-makin-populer-ini-alasan-ramai-pindah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240725070308-17-557427/btc-tak-sanggup-tembus-us--66000-pasar-kripto-lanjut-turun',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240724055509-128-557099/ramalan-robert-kiyosaki-bitcoin-tembus-us-100000-jika-trump-menang',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240724045245-17-557096/pasar-kripto-lanjut-ambruk-ini-penyebabnya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240723130306-17-556895/memecoin-biden-anjlok-66-usai-mundur-dari-pilpres-kamala-naik-133',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240723063541-17-556782/pasar-tak-antusias-sambut-etf-ethereum-kripto-terkoreksi',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240722150506-37-556666/donald-trump-jadi-bintang-harga-tiket-konferensi-kripto-tembus-rp13-m',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240722150730-17-556669/jadi-calon-presiden-as-gantikan-biden-segini-harta-kamala-harris',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240722105551-17-556535/joe-biden-dipastikan-tak-lawan-trump-kripto-menghijau',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240721134821-17-556379/harga-kripto-terbang-dalam-sepekan-ternyata-gegara-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240719165907-37-556163/kripto-sampai-pinjol-setor-pajak-rp38-t-januari-juni-2024',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240719060112-17-555910/polemik-joe-biden-vs-donald-trump-bikin-pasar-kripto-bergerak-mix',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240718060655-17-555545/bursa-saham-as-anjlok-pasar-kripto-ikut-ambruk-di-zona-merah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240717185432-8-555502/kominfo-blokir-akun-binance-hingga-turis-tewas-di-hotel-bangkok',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240717172710-8-555477/kominfo-blokir-medsos-kripto-hingga-spekulasi-penembakan-trump',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240717120732-17-555342/kominfo-blokir-medsos-kripto-binance-kucoin-bappebti-buka-suara',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240717090356-17-555266/harga-bitcoin-tembus-rp-105-miliar-pasar-kripto-pesta-pora',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240715063019-17-554525/etf-bitcoin-spot-bawa-masuk-rp-5-triliun-kripto-kompak-melaju',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240711145452-17-553773/terkuak-pasar-online-kamboja-fasilitasi-penipuan-kripto-rp-178-t',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240708093440-17-552540/smart-money-vs-dumb-money-beda-arah-kripto-jatuh-signifikan',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240704122335-17-551740/siap-siap-intip-10-koin-kripto-berpotensi-cuan-di-2024',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240702180010-57-551222/arfadia-pionir-layanan-seo-dan-pemasaran-digital-kripto-di-indonesia',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240628191516-17-550365/islamic-coin-dapat-fatwa-halal-mui-kenya-ini-alasannya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240628080134-17-550091/dibanding-bitcoin-warren-buffett-lebih-pilih-aset-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240621112837-17-548110/si-kembar-winklevoss-kecam-presiden-as-kripto-kebakaran',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/mymoney/20240624195948-72-548972/platform-investasi-ini-gelar-fase-ii-airdrop-gratis-3-juta-token',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240619101805-17-547383/produk-gabungan-etf-btc-eth-bikin-geger-pasar-kripto-pesta-pora',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240614084346-17-546553/the-fed-pangkas-buka-cuma-1-kali-kripto-lanjut-terkoreksi',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240611091811-17-545534/sebut-kripto-koin-judi-warren-buffett-punya-alasan-sendiri',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240610085235-17-545143/tekanan-jual-mereda-kripto-mulai-naik-hari-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240507064019-17-536230/regulator-kembali-berulah-bitcoin-turun-tapi-solana-menghijau',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240430091703-17-534527/sambut-altcoin-season-20-simak-3-koin-kripto-yang-bisa-bawa-cuan',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240429083057-17-534153/inflasi-as-masih-panas-bitcoin-turun-055-ethereum-di-zona-hijau',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240428144117-37-534076/waspada-jangan-install-aplikasi-android-ini-bisa-bobol-rekening',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/mymoney/20240425155629-74-533440/video-marak-pencucian-uang-lewat-kripto-gimana-hukum-mengungkapnya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240423175409-37-532742/menelisik-prospek-cuan-di-tengah-momentum-halving-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240419143043-4-531712/diungkap-ppatk-begini-modus-tppu-lewat-kripto-yang-disinggung-jokowi',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240419124616-17-531653/jokowi-warning-pencucian-uang-di-kripto-rp139-t-ppatk-lakukan-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240419105454-4-531615/ppatk-ungkap-pencucian-uang-lewat-kripto-sebesar-rp-800-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20240417184523-8-531130/video-jokowi-ungkap-dugaan-pencucian-uang-kripto-rp-139-t',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240417172546-17-531107/jokowi-warning-pencucian-uang-di-kripto-bos-ojk-mau-lakukan-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240417153821-37-531061/jokowi-beri-peringatan-modus-pencucian-uang-baru-lewat-kripto-ngeri',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240414201017-128-530333/kripto-jadi-korban-konflik-iran-vs-israel-market-cap-ambles-rp2000-t',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240411180744-128-529856/kripto-cs-patut-waspada-indeks-dolar-perkasa-efek-inflasi-as-panas',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240410224447-17-529723/bappebti-keluarkan-aturan-baru-buat-pedagang-aset-kripto-ri-simak',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20240411091258-128-529759/kripto-konsisten-naik-sepanjang-2024-apa-sebabnya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240410194107-17-529712/ekonomi-amerika-makin-panas-dampaknya-ke-aset-ini',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240328091939-17-526193/investor-rusuh-tarik-dana-di-kucoin-kripto-kompak-berdarah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240327092909-17-525810/bitcoin-minggir-3-aset-kripto-ini-bisa-bawa-cuan-besar',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240322134131-17-524478/bitcoin-tembus-rekor-ini-tips-buat-para-calon-investor',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240319072824-17-523087/usdt-dihentikan-bagi-pengguna-zona-eropa-kripto-kompak-merah',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/tech/20240315142843-37-522306/sosok-pencipta-bitcoin-satoshi-nakamoto-terbongkar-ini-faktanya',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240313103348-17-521498/kripto-dan-binance-dituduh-jadi-biang-kerok-mata-uang-nigeria-ambruk',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240205125303-17-511936/ini-modus-pencucian-uang-paling-populer-di-indonesia',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240124110317-17-508551/warren-buffett-sebut-kripto-koin-judi',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20240104124216-17-502720/catat-ini-7-faktor-yang-mempengaruhi-harga-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/research/20231230105347-128-501456/lika-liku-kripto-di-2023-btc-sentuh-rekor-15-tahun-terakhir',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20231227183128-17-500713/cerita-3-bank-kripto-kompak-bangkrut-dalam-sepekan',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20231214132546-17-497356/2-sentimen-ini-diprediksi-bawa-angin-segar-bagi-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20231130141541-17-493370/gegara-ftx-mulai-likuidasi-aset-pasar-kripto-ambruk',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/news/20231129194807-4-493134/dari-mana-sumber-dana-hamas-ini-hasil-penyelidikan-israel',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20231124134456-17-491752/duh-kasus-cuci-uang-bos-binance-bikin-investor-kripto-panik',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
    {
        'url': 'https://www.cnbcindonesia.com/market/20231122184727-20-491219/lengkap-begini-jatuh-bangun-harga-bitcoin-sejak-2009-2023',
        'title_tag': 'h1',
        'title_class': 'mb-4 text-32 font-extrabold',        
        'date_tag': 'div',
        'date_class': 'text-cm text-gray',
        'content_tag': 'div',
        'content_class': 'detail-text'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/cnbcindonesia.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

Data has been saved to ./preprocessed/cnbcindonesia.csv
                                                 Title  \
0    OJK Bekukan 1.001 Entitas Ilegal, 850 Ternyata...   
1    Bitcoin Diguyur Banyak Sentimen Positif, Pasar...   
2    Kisah Negara Kecil Mendadak Kaya Raya Berkat B...   
3    Ngenes! Sosok Ini Dapat Durian Runtuh Rp163 M,...   
4       Kripto Turun Berjamaah, Pasar Khawatir Hal Ini   
..                                                 ...   
121  2 Sentimen Ini Diprediksi Bawa Angin Segar Bag...   
122  Gegara FTX Mulai Likuidasi Aset, Pasar Kripto ...   
123  Dari Mana Sumber Dana Hamas? Ini Hasil Penyeli...   
124  Duh, Kasus Cuci Uang Bos Binance Bikin Investo...   
125  Lengkap! Begini Jatuh Bangun Harga Bitcoin Sej...   

                       Date                                            Summary  
0      19 August 2024 14:20  Jakarta, CNBC Indonesia- Satuan Tugas Pemberan...  
1      19 August 2024 09:45  Jakarta, CNBC Indonesia -Pasar kripto mengalam...  
2   